In [2]:
import numpy
import pandas as pd
import xlwt
from xlwt import Workbook
from __future__ import print_function, division
from data_loader import ImageFolder
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.utils as utils

In [6]:
model = models.vgg16(pretrained=False)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_features=num_ftrs,out_features=10)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [15]:
model = models.vgg16(pretrained=False)

num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(in_features=num_ftrs,out_features=10)

# model.load_state_dict(torch.load('/Users/suchethas/Documents/model_imageclass10.pth', map_location='cpu'))
modified_model = nn.Sequential(*list(model.features.children()))

In [16]:
print(modified_model)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [17]:
model = models.vgg16(pretrained=True)

#Removing the classifier layer
modified_model = nn.Sequential(*list(model.features.children()))

def make_layers_decoder(cfg_decoder, batch_norm=False):
    layers_decoder = []
    s = [512, 512, 256, 128, 64]
    si = 0
    inn_channels = s[si]
    for v in range(len(cfg_decoder)):
        if cfg_decoder[v] == 'M':
            trans_conv2d = nn.ConvTranspose2d(inn_channels, s[si], kernel_size=2, stride = 2, padding=0)
            layers_decoder += [trans_conv2d, nn.ReLU(inplace=True)]
            si = si+1
        else:
            trans_conv2d = nn.ConvTranspose2d(inn_channels, cfg_decoder[v], kernel_size=3, padding=1)
            if batch_norm:
                if v<=len(cfg_decoder)-2:
                    if cfg_decoder[v+1]=='M':
                        #layers_decoder += [trans_conv2d, nn.BatchNorm2d(cfg_decoder[v])]
                        layers_decoder += [trans_conv2d]
                    else:
                        layers_decoder += [trans_conv2d, nn.BatchNorm2d(cfg_decoder[v]), nn.ReLU(inplace=True)]
                else:
                    layers_decoder += [trans_conv2d]
                #layers_decoder += [trans_conv2d, nn.BatchNorm2d(cfg_decoder[v]), nn.ReLU(inplace=True)]
            else:
                if v<=len(cfg_decoder)-2:
                    if cfg_decoder[v+1]=='M':
                        layers_decoder += [trans_conv2d]
                    else:
                        layers_decoder += [trans_conv2d, nn.ReLU(inplace=True)]
                else:
                    layers_decoder += [trans_conv2d]
            inn_channels = cfg_decoder[v]
    return nn.Sequential(*layers_decoder)

cfg_decoder = {
    'D' : ['M', 512, 512, 512, 'M', 512, 512,  256, 'M', 256, 256,  128, 'M', 128,  64, 'M', 64, 3],
}

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = modified_model
        self.decoder = make_layers_decoder(cfg_decoder['D'])
        
    def forward(self,x):
        x = modified_model(x)
        #print(x)
        x = self.handlesequential_decoder(self.decoder,x)
        return x
    
    def handlesequential_decoder(self,layer,x):
        for i,module in enumerate(layer.children()):
                x = module(x)
        return x
    
ae = Autoencoder()

In [18]:
print(ae)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17):